In [2]:
import numpy as np
import pandas as pd

In [3]:
hechos = pd.read_excel('homicidios.xlsx')
victimas =pd.read_excel('homicidios.xlsx', sheet_name='VICTIMAS')

## KPI 1: Tasa de homicidios en siniestros viales por semestre

Definimos a la tasa de homicidios en siniestros viales como el número de víctimas fatales en accidentes de tránsito por cada 100,000 habitantes en un área geográfica durante un período de tiempo específico. Su fórmula es: (Número de homicidios en siniestros viales / Población total) * 100,000

**Objetivo**
- Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior

In [4]:
def asignar_semestre(mes):
    if 1 <= mes <= 6:
        return 1
    elif 7 <= mes <= 12:
        return 2
    else:
        return None

In [34]:
hechos['Semestre'] = hechos['MM'].apply(asignar_semestre)

In [33]:
victimas['Semestre'] = victimas['MM'].apply(asignar_semestre)

In [7]:
victimas.head(2
            )

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,Semestre
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00,1
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00,1


In [35]:
df_victimas_semestre= hechos.groupby(['AAAA', 'Semestre'])['N_VICTIMAS'].sum().reset_index()
df_victimas_semestre

,AAAA,Semestre,N_VICTIMAS
0,2016,1,65
1,2016,2,81
2,2017,1,69
3,2017,2,71
4,2018,1,70
5,2018,2,79
6,2019,1,57
7,2019,2,47
8,2020,1,31
9,2020,2,50


La poblacion total se obtiene del resultado del censo del 2022 de la pagina Argentina.gob.ar

In [9]:
poblacion_total = 3120612

In [36]:
df_victimas_semestre['Tasa de Homicidios'] = (df_victimas_semestre['N_VICTIMAS'] / poblacion_total) * 100000
df_victimas_semestre


,AAAA,Semestre,N_VICTIMAS,Tasa de Homicidios
0,2016,1,65,2.082925
1,2016,2,81,2.595645
2,2017,1,69,2.211105
3,2017,2,71,2.275195
4,2018,1,70,2.243150
5,2018,2,79,2.531555
6,2019,1,57,1.826565
7,2019,2,47,1.506115
8,2020,1,31,0.993395
9,2020,2,50,1.602250


In [27]:
df_victimas_semestre['Cambio_tasa'] = df_victimas_semestre['Tasa de Homicidios'].pct_change() * 100
df_victimas_semestre

,AAAA,Semestre,N_VICTIMAS,Tasa de Homicidios,Cambio_tasa
0,2016,1,65,2.082925,NaN
1,2016,2,81,2.595645,24.615385
2,2017,1,69,2.211105,-14.814815
3,2017,2,71,2.275195,2.898551
4,2018,1,70,2.243150,-1.408451
5,2018,2,79,2.531555,12.857143
6,2019,1,57,1.826565,-27.848101
7,2019,2,47,1.506115,-17.543860
8,2020,1,31,0.993395,-34.042553
9,2020,2,50,1.602250,61.290323


In [23]:
df_victimas_semestre.to_excel('Tasa homicidios.xlsx', index = False)

## KPI2: Cantidad de accidentes mortales de motociclistas en siniestros viales

Definimos a la cantidad de accidentes mortales de motociclistas en siniestros viales como el número absoluto de accidentes fatales en los que estuvieron involucradas víctimas que viajaban en moto en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes mortales con víctimas en moto es: (Número de accidentes mortales con víctimas en moto en el año anterior - Número de accidentes mortales con víctimas en moto en el año actual) / (Número de accidentes mortales con víctimas en moto en el año anterior) * 100

**Objetivo**
- Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior

In [38]:
df_victimas_motos = victimas[victimas['VICTIMA'] == 'MOTO']
df_victimas_motos.head(3)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,Semestre
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00,1
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00,1
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD,1


In [39]:
df_victimas_motos = df_victimas_motos.groupby('AAAA')['ID_hecho'].count().reset_index(name='Cantidad de victimas')
df_victimas_motos

,AAAA,Cantidad de victimas
0,2016,65
1,2017,56
2,2018,57
3,2019,50
4,2020,29
5,2021,46


In [41]:
df_victimas_motos['Evolucion'] = ((df_victimas_motos['Cantidad de victimas'].shift(1) - df_victimas_motos['Cantidad de victimas']) / df_victimas_motos['Cantidad de victimas'].shift(1)) * 100
df_victimas_motos.head()

,AAAA,Cantidad de victimas,Evolucion
0,2016,65,NaN
1,2017,56,13.846154
2,2018,57,-1.785714
3,2019,50,12.280702
4,2020,29,42.000000


In [48]:
df_victimas_motos['Diferencia en Porcentaje'] = df_victimas_motos['Evolucion'].pct_change()
df_victimas_motos

,AAAA,Cantidad de victimas,Evolucion,Diferencia en Porcentaje
0,2016,65,NaN,NaN
1,2017,56,13.846154,NaN
2,2018,57,-1.785714,-1.128968
3,2019,50,12.280702,-7.877193
4,2020,29,42.000000,2.420000
5,2021,46,-58.620690,-2.395731


In [49]:
df_victimas_motos.to_excel('Cantidad victimas motos.xlsx', sheet_name='Filtro motos', index = False)

## KPI3: Tasa de homicidios en siniestros viales por año

Definimos a la tasa de homicidios en siniestros viales como el número de víctimas fatales en accidentes de tránsito por cada 100,000 habitantes en un área geográfica durante un período de tiempo específico. Su fórmula es: (Número de homicidios en siniestros viales / Población total) * 100,000

**Objetivo**
- Reducir en un 15% la tasa de homicidios en siniestros viales de el ultimo año, en CABA, en comparación con la tasa de homicidios en siniestros viales del año anterior

In [50]:
df_victimas_año= hechos.groupby(['AAAA'])['N_VICTIMAS'].sum().reset_index()
df_victimas_año

,AAAA,N_VICTIMAS
0,2016,146
1,2017,140
2,2018,149
3,2019,104
4,2020,81
5,2021,97


In [51]:
df_victimas_año['Homicidios por año'] = (df_victimas_año['N_VICTIMAS'] / poblacion_total) * 100000
df_victimas_año

,AAAA,N_VICTIMAS,Homicidios por año
0,2016,146,4.678569
1,2017,140,4.486299
2,2018,149,4.774704
3,2019,104,3.332680
4,2020,81,2.595645
5,2021,97,3.108365


In [52]:
df_victimas_año['Cambio'] = df_victimas_año['Homicidios por año'].pct_change() * 100
df_victimas_año

,AAAA,N_VICTIMAS,Homicidios por año,Cambio
0,2016,146,4.678569,NaN
1,2017,140,4.486299,-4.109589
2,2018,149,4.774704,6.428571
3,2019,104,3.332680,-30.201342
4,2020,81,2.595645,-22.115385
5,2021,97,3.108365,19.753086


In [55]:
df_victimas_año.to_excel('Homicidios por año.xlsx', sheet_name='Victimas año', index = False)